# 5 - Conclusões

Resumimos aqui as principais conclusões alcançadas no trabalho de modelagem para predição de resultados de testes de COVID.

In [ ]:
# para carregar a base de dadas limpa
import pickle

import numpy as np
import pandas as pd
from scipy import stats as spst

from pprint import pprint

# gráficos
import seaborn as sns
from matplotlib import rcParams, pyplot as plt

# parâmetros do matplotlib
# essencialmente, para deixar os gráficos maiores por padrão
rcParams['figure.dpi'] = 120
rcParams['figure.figsize'] = (10, 8)

# warnings
import warnings
warnings.filterwarnings("ignore")

# pacote com funções para análise desse projeto
import os
cwd = os.getcwd()
os.chdir("../")
import scripts.plots as splt, scripts.metrics as smetrics
os.chdir(cwd)

## Importação dos modelos fitados

Vamos importar os modelos já fitados porém com parâmetros padrão.

In [ ]:
with open(r'../models/modelo_default.model', 'rb') as modelfile:
    pickler = pickle.Unpickler(file = modelfile)
    modelos_import = pickler.load()

In [ ]:
modelos = modelos_import['modelo']

X_train, X_test, y_train, y_test = modelos_import['train_test_split']

In [ ]:
covid = modelos_import['base']

covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 485731 to 53605
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   sex                  50000 non-null  category
 1   patient_type         50000 non-null  category
 2   pneumonia            50000 non-null  category
 3   age                  50000 non-null  int8    
 4   pregnancy            50000 non-null  category
 5   diabetes             50000 non-null  category
 6   copd                 50000 non-null  category
 7   asthma               50000 non-null  category
 8   inmsupr              50000 non-null  category
 9   hypertension         50000 non-null  category
 10  other_disease        50000 non-null  category
 11  cardiovascular       50000 non-null  category
 12  obesity              50000 non-null  category
 13  renal_chronic        50000 non-null  category
 14  tobacco              50000 non-null  category
 15  contact_other_

In [ ]:
covid.head()

,sex,patient_type,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res
id,,,,,,,,,,,,,,,,,
485731,1,1,0,57,0,0,0,0,0,1,0,0,0,0,0,1,0
278436,0,0,0,87,0,0,0,0,0,1,1,0,0,0,0,-1,0
372264,1,1,0,28,0,0,0,0,0,0,0,0,0,0,0,1,0
321682,1,1,0,48,0,0,0,0,0,0,0,0,0,0,0,0,0
247228,1,1,0,9,0,0,0,0,0,0,0,0,0,0,0,1,0


## Análise exploratória da base de dados

Antes do ajuste dos modelos, é importante conhecermos os dados: o que significa cada campo, o tipo dos dados (categóricos, numéricos, ordinais, texto etc), a ordem de grandeza dos valores, como eles se relacionam entre si etc.

Para tal, conduzimos uma análise exploratória dos dados de testes de COVID.

O único campo numérico é a idade; os outros são todos categóricos binários (homem/mulher, sim/não, positivo/negativo etc).

Embora haja tecnicamente 32.4 milhões de combinações dos valores únicos combinados de todos os campos, vimos que as colunas são altamente correlacionadas. 82% das correlações 2 a 2 foram consideradas estatisticamente significativas.

Eliminamos da análise os campos `intubed` e `icu` da análise, visto que nestes campos o preenchimento é muito inconsistente (acima de 70% de dados faltantes), de forma que em quaisquer novos provavelmente estes campos também não estarão preenchidos, o que os torna de pouco valor para o modelo.

Vimos que, entre os pacientes que foram examinados e liberados para casa, não há muita diferença de idade entre os que tiveram um resultado positivo ou um negativo (entre 25 e 35 anos, aproximadamente). Já para os que foram internados, os pacientes que tiveram um resultado positivo de COVID são mais idosos e com dispersão muito menor que os com resultado negativo.

Inicialmente formulammos uma hipótese em que o paciente seria internado primariamente por presença de comorbidades evidentes, seja visualmente (*e.g.* obesidade) ou em exames básicos (*e.g.* pneumonia).

No entanto, essa hipótese não se confirmou; a presença de comorbidades responde por apenas 12% da correlação com o *status*  de internação.

Esperávamos que as comorbidades tivessem grande relação com o resultado do teste de COVID. Essa hipótese se confirmou: mesmo ajustando o nível de significância para considerar múlitplas comparações, chegamos aà conclusão que relação entre as comorbidades e o resultado do teste são estatisticamente significativas.

No entanto, essas relações são diferentes a depender da faixa etária:

* **Crianças até 10 anos**: estão mais vulneráveis ao acometimento por COVID se tem pneumonia ou problemas relacionados à obesidade (excesso de peso ou diabetes);
* **Adolescentes, adultos até 60 anos e idosos entre 60 e 80 anos**: estão mais vulneráveis ao contágio se tem qualquer comorbidade no conjunto de dados; e
* **Idosos com mais de 80 anos**: Estão mais vulneráveis ao acometimento por COVID se tem doenças pulmonares, se são imunossuprimidos ou se tem outras doenças.

No geral, as maiores comorbidades associadas a um teste positivo de COVID são pulmonares ou com pacientes imunossuprimidos.

Notamos também que, 
* com exceção de crianças até 10 anos, **a prática regular do fumo está relacionada ao acometimento por COVID**; e
* em todas as faixas etárias, **a presença de problemas relacionados a obesidade (excesso de peso ou diabetes) está associada ao contágio por COVID**. A exceção são os muito idosos; uma razão para isso é o fato de os obesos não chegarem a idades mais avançadas em números suficientes.

A tabela abaixo mostra o teste de hipótese em que a hipótese nula $H_0$ é **não há nenhuma relação entre o grupo de comorbidades e o resultado do teste de COVID para cada faixa etária**.

Reijeitamos $H_0$?

|          Comorbidade | criança até 10 anos | adulto (entre 10 e 60 anos) | idoso (entre 60 e 80 anos) | muito idoso (acima de 80 anos) |
|----------------:|:-------------------:|:---------------------------:|:--------------------------:|:------------------------------:|
|         Coração |         Não         |             Sim             |             Sim            |               Não              |
|          Pulmão |         Sim         |             Sim             |             Sim            |               Sim              |
|       Obesidade |         Sim         |             Sim             |             Sim            |               Não              |
| Imunossupressão |         Não         |             Sim             |             Sim            |               Sim              |
|          Outras |         Não         |             Não             |             Sim            |               Sim              |

Uma outra variável potencialmente importante para o modelo é o sexo. Homens tem mais resultados positivos de COVID que mulheres (diferença estatisticamente significativa):

![testes chisq entre sexo e resultado teste de covid](../assets/sexo_resultado_chisq.png)

Por fim, uma última variável que pode se mostrar relevante é se o paciente teve contato com outras pessoas comprovadamente com COVID (`contact_other_covid`). Ela tem relação com alguns outros campos.

## Construção do modelo

Construímos o modelo blaha

## Otimização dos modelos

Cada modelo de aprendizado de máquina é construído em cima de alguns parâmetros. Por exemplo, no modelo de regressão logística, há um parâmetro regulando a presença ou não do intercepto (`fit_intercept`) e um outro controlando a regularização do modelo, *i.e.* a penalização para muitos coeficientes (`penalty`), entre outros. Estes são chamados ***hiperparâmetros***.

A otimização do modelo consiste na escolha dos hiperparâmetros de forma a maximizar/minimizar alguma métrica. Neste caso, para sermos consistentes com o estudo realizado na parte de modelagem, vamos testar várias combinações de hiperparâmetros e ranqueá-las através do *Brier score*. Um modelo se adere mais ao resultado se essa métrica for menor que a de um outro modelo; logo, minimizamos o *Brier score* das combinações de hiperparâmetros. Levamos em conta também a área sob a curva ROC.

Dos modelos de aprendizado de maquina que testamos para predizer o resultado dos testes de COVID, dois performaram muito bem com parâmetros *default*: regressão logística e *XGBoost*.

Para cada um deles, realizamos uma busca no espaço de hiperparâmetros para determinar quais eram mais aplicáveis ao problema em questão, *i.e.* qual conjunto performaria melhor nas métricas escolhidas (*Brier score* e AUROC).

Como podemos ver na tabela abaixo, o modelo de regressão logística começou com métricas ligeiramente melhores, mas com parâmetros otimizados o modelo XGBoost performou melhor:

***TABELA!***

![curva roc otimizada](../assets/roc_otimizado.png)

Por esse motivo, escolhemos o modelo XGBoost com os seguintes parâmetros:

- `base_score`: 0.5,
- `booster`: 'gbtree',
- `colsample_bylevel`: 1,
- `colsample_bynode`: 1,
- `colsample_bytree`: 1,
- `criterion`: 'mae',
- `enable_categorical`: False,
- `eval_metric`: 'logloss',
- `gamma`: 0,
- `gpu_id`: -1,
- `importance_type`: None,
- `interaction_constraints`: '',
- `learning_rate`: 0.1,
- `max_delta_step`: 0,
- `max_depth`: 3,
- `max_features`: 'log2',
- `min_child_weight`: 1,
- `missing`: nan,
- `monotone_constraints`: '()',
- `n_estimators`: 100,
- `n_jobs`: 8,
- `num_parallel_tree`: 1,
- `objective`: 'binary:logistic',
- `predictor`: 'auto',
- `random_state`: 42,
- `reg_alpha`: 0,
- `reg_lambda`: 1,
- `scale_pos_weight`: 1,
- `subsample`: 1,
- `tree_method`: 'exact',
- `use_label_encoder`: True,
- `validate_parameters`: 1,
- `verbosity`: None

Conduzimos ainda uma análise de otimização do threshold de decisão. Entendemos que um falso negativo na predição de um resultado de teste de COVID (predizer que o resultado dará negativo, mas o resultado dar positivo) é mais prejudicial que um falso positivo; no primeiro caso, o paciente será liberado e pode infectar outras pessoas, facilitando a transmissão; já no segundo, o paciente se isolará sem necessidade, porém sem maiores consequências.

Em outras palavras, a métrica *recall* (dentre os resultados de testes positivos/negativos, quantos acertamos?) é mais importante para a otimização do *threshold* que a precisão (dentre os resultamos que predizemos ser positivos, quantos acertamos?).

Sendo assim, utilizamos duas métricas de referência:

* **Custo estimado**: estimamos que o custo de um falso negativo é 5x maior que o custo de um falso positivo; e
* **F-$\beta$ score**: como priorizamos o *recall* em detrimento da precisão, utilizamos um $\beta = 1.5$

Conforme observamos no gráfico...

![otimizacao de threshold](../assets/custo_threshold.png)

...um *threshold* $\theta = 24\%$ parece ser o melhor para limiar de tomada de decisão.